In [1]:
from preprocess.arpit_v2 import *
from preprocess.preprocess_v2 import *
from preprocess.preprocess_v2 import preprocess
import os
import inspect
import time
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import pickle
from models.LDA_multi_level import lda_model_multi_level
from models.LDA_single_level import lda_model_single_level
print('------------------------------------------------------')
print('- Imports Done')

c:\users\ajain7\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


------------------------------------------------------
- Imports Done


In [2]:
# DATA
# Note that raw docs is a numpy array. 
# Example element is: 
# 'Logical Disk Free Space is low, Description: The disk C: on computer sjcphxstg02.strykercorp.com is running out of disk space. The values that exceeded the thre'
# data_file_string = 'short_description.pkl'
data_file_string = 'data.pkl'
data_file = os.path.join(os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))),'data',data_file_string)
raw_docs = pickle.load(open(data_file,'rb'))
print('- Imported Data')

- Imported Data


In [3]:
# PRE-PROCESSING
preprocess_steps_and_order = {
	'make_lowercase': [True],
	'punctuation_removal':[True],
	'whitespace_removal': [True],
	'store_alphanumeric': [False],
	'pos_removal_nltk': [True],
	'tokenization_nltk': [False],
	'lemmatization_tokenization_spacy': [True],
	'stopwords_removal_nltk': [True],
	'stopwords_removal_spacy': [False],
	'make_bigrams_gensim':[True, {'make_bigrams_gensim': True, 'bigrams_min_count': 10, 'bigrams_threshold': 10}],
	'make_trigrams_gensim':[True, {'make_trigrams_gensim': True, 'trigrams_min_count': 10, 'trigrams_threshold': 10}],
	'min_max_length_removal':[False, {'min_max_length_removal': False, 'mmlr_min_len': 3, 'mmlr_max_len': 50, 'mmlr_deacc': False}]
	}

preprocess_functions = {
	'make_lowercase': make_lowercase,
	'punctuation_removal': punctuation_removal,
	'whitespace_removal': whitespace_removal,
	'store_alphanumeric': store_alphanumeric,
	'pos_removal_nltk': pos_removal_nltk,
	'tokenization_nltk': tokenization_nltk,
	'lemmatization_tokenization_spacy': lemmatization_tokenization_spacy,
	'stopwords_removal_nltk': stopwords_removal_nltk,
	'stopwords_removal_spacy': stopwords_removal_spacy,
	'make_bigrams_gensim': make_bigrams_gensim,
	'make_trigrams_gensim': make_trigrams_gensim,
	'min_max_length_removal': min_max_length_removal
	}

In [4]:
# MODELS
models_dict = {
	'LDA_single_level': lda_model_single_level,
	'LDA_multi_level': lda_model_multi_level,
}

In [5]:
# SPECIFICATIONS
specifications = {
	# 'model':'LDA_single_level', # Can be LDA_multi_level
	'level':2,
	'num_topics_list_level_1':[5,10,15,20,25],
	'num_topics_list_level_2':[3,5,8,11],
	'num_topics_list_level_3':[1,2,3,4,5],
	'coherence':'c_v',
	'need_best_topic': True,
	'model_selection_metric':'coherence', # or 'perplexity',
	'debug':False,
}

In [6]:
print('*****************************************************')
print('- Starting preprocessing')
dictionary, corpus, doc_list = preprocess(
								raw_docs = raw_docs, 
								preprocess_functions = preprocess_functions, 
								preprocess_steps_and_order = preprocess_steps_and_order, 
								debug=False)

*****************************************************
- Starting preprocessing

       ##### Lowercasing Done! Time Taken -  0.028003931045532227

       ##### Punctuation removed! Time Taken -  0.2020106315612793

       ##### Whitespace removed! Time Taken -  0.08400464057922363

       ##### POS Removal Done! Time Taken -  40.13323187828064

       ##### Lemmatization and Tokenization Done using Spacy! Time Taken -  76.04995536804199

       ##### Stopwords Removed using NLTK! Time Taken -  0.3080465793609619

       ##### Bi-Grams made using Gensim! Time Taken -  2.110252857208252

       ##### Tri-Grams made using Gensim! Time Taken -  1.9426908493041992
~~~ pre-processing done in  120.91414070129395
 
- Creating dictionary and corpus


In [7]:
print('*****************************************************')
print('- Starting model training')
lda_dict = lda_model_single_level(
					dictionary = dictionary,
					corpus = corpus,
					doc_list = doc_list,
					num_topics_list_level_1 = specifications['num_topics_list_level_1'], 
					coherence = specifications['coherence'],
					debug = specifications['debug'],
					need_best_topic = specifications['need_best_topic'],
					model_selection_metric = specifications['model_selection_metric']
					)

*****************************************************
- Starting model training
 
Sample data point:  ['dataset', 'transaction', 'credit', 'card', 'september', 'cardholder', 'dataset', 'transaction', 'day', 'fraud', 'transaction', 'dataset', 'class', 'fraud', 'account', 'transaction', 'input', 'variable', 'result', 'transformation', 'confidentiality', 'issue', 'feature', 'background', 'datum', 'feature', 'v28', 'component', 'pca', 'feature', 'pca', 'transaction', 'transaction', 'dataset', 'feature', 'amount', 'transaction', 'amount', 'feature', 'dependant', 'cost', 'feature', 'class', 'response', 'variable', 'value', 'case', 'fraud', 'class', 'imbalance', 'ratio', 'accuracy', 'area', 'precision', 'recall', 'curve', 'confusion', 'matrix', 'accuracy', 'classification', 'dataset', 'research', 'collaboration', 'worldline', 'machine', 'group', 'ac', 'libre', 'bruxelle', 'datum_mining', 'fraud', 'detection', 'detail', 'project', 'topic', 'http', 'mlg', 'brufence', 'http', 'mlg', 'dal', 'pozz

In [13]:
# pyLDAvis.enable_notebook()
# print(lda_dict['best_topic'])
# print(lda_dict['coherence_score'])
# print(lda_dict['perplexity_score'])
# visualization = pyLDAvis.gensim.prepare(lda_dict['best_lda_model'], lda_dict['corpus'], lda_dict['dictionary'])
# pyLDAvis.save_html(visualization, 'lda.html')
# visualization

5
0.47154971390590406
-8.58700882191759


In [ ]:
print('*****************************************************')
print('- Starting model training')
lda_level_1, lda_level_2 = lda_model_multi_level(
					level = specifications['level'],
					dictionary = dictionary,
					corpus = corpus,
					doc_list = doc_list,
					coherence = specifications['coherence'],
					debug = specifications['debug'],
					need_best_topic = specifications['need_best_topic'],
					model_selection_metric = specifications['model_selection_metric'],
					num_topics_list_level_1 = specifications['num_topics_list_level_1'], 
					num_topics_list_level_2 = specifications['num_topics_list_level_2'], 
					)